In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf

In [8]:
#importamos los ratings para cada lugar
Y_data = pd.read_csv('ratings_place.csv', sep=",", header = None).values
Y_data.shape

(34, 57)

In [22]:
#importamos la lista de lugares
places_list = pd.read_csv('locations.csv', sep=",", skiprows =[0],usecols=[0], header = None).values
places_list.shape

(34, 1)

In [24]:
#Logica del filtrado colaborativo para el minado de reglas de asociacion

Y = tf.placeholder(tf.float32)

X = tf.Variable(tf.truncated_normal([34, 100], stddev=0.1))
Theta = tf.Variable(tf.truncated_normal([57, 100], stddev=0.1))

#####
R = tf.cast(tf.not_equal(Y, 0), tf.float32)

Y_mean = (tf.reduce_sum(Y, axis = 1) / tf.reduce_sum(R, axis = 1))

Y_norm = Y - tf.reshape(Y_mean,[-1,1])

####

cost = 0.5 * tf.reduce_sum(R * (tf.square(tf.matmul(X, tf.transpose(Theta)) - Y_norm)))

train_step = tf.train.AdamOptimizer(1e-3).minimize(cost)

#predicciones
prediction = tf.matmul(X, tf.transpose(Theta)) + tf.reshape(Y_mean,[-1,1])

#creamos la sesion de tf
sess = tf.InteractiveSession()
tf.global_variables_initializer().run()

#entrenamiento
for i in range(10000):
    if i % 100 == 0:
        print('step %d, training cost %g' % (i, cost.eval(feed_dict={Y: Y_data})))
    sess.run(train_step, feed_dict={Y: Y_data})

Y_pred = prediction.eval(feed_dict={Y: Y_data})
print('Predicted Y:\n', Y_pred)

/home/jose/.local/lib/python3.7/site-packages/tensorflow/python/client/session.py:1702: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


step 0, training cost 574.74
step 100, training cost 172.99
step 200, training cost 21.9527
step 300, training cost 1.38284
step 400, training cost 0.117048
step 500, training cost 0.0136412
step 600, training cost 0.0016182
step 700, training cost 0.000194502
step 800, training cost 2.41539e-05
step 900, training cost 3.01149e-06
step 1000, training cost 3.54678e-07
step 1100, training cost 3.75097e-08
step 1200, training cost 3.65561e-09
step 1300, training cost 4.05938e-10
step 1400, training cost 8.75298e-11
step 1500, training cost 3.73755e-11
step 1600, training cost 2.31422e-11
step 1700, training cost 1.64008e-11
step 1800, training cost 1.24389e-11
step 1900, training cost 9.97489e-12
step 2000, training cost 8.34859e-12
step 2100, training cost 6.75171e-12
step 2200, training cost 5.57794e-12
step 2300, training cost 4.66129e-12
step 2400, training cost 4.36428e-12
step 2500, training cost 4.36261e-12
step 2600, training cost 3.78726e-12
step 2700, training cost 4.26277e-12
s

In [25]:
Y_not_seen = (Y_data==0) * Y_pred
recommend = np.argsort(-Y_not_seen, axis=0)

# Muestra las k mejores recomendaciones para el usuario j
k = 15
j = 1

for i in range (k):
    print(places_list[recommend[i,j],0])

1
32
31
30
29
28
27
26
25
24
23
22
21
20
19


In [27]:
#ahora cargamos los perfiles por usuario en un df

array([[ 0,  0,  0, ...,  0,  0,  0],
       [31, 31, 31, ..., 31, 31, 31],
       [30, 30, 30, ..., 30, 30, 30],
       ...,
       [14, 14, 14, ..., 14, 14, 14],
       [ 8,  8,  8, ...,  8,  8,  8],
       [33, 33, 33, ..., 33, 33, 33]])

In [28]:
sess.close()